Find the leading scorer by zone on the court. Plot the zones and players' names.

Plot the shooting charts of teams and/or players

In [1]:
import sys
from matplotlib import pyplot as plt
from euroleague_api.shot_data import ShotData

sys.path.append("../utils/")
from shot_chart_plots import plot_scatter, plot_leading_scorers_by_zone, fg_perc_hex_heatmap
from get_new_zones import hexagon_zones, corner_three_zones
# %matplotlib

plt.close("all")

In [2]:
def get_leading_scorer_by_zone(df, zone_col="ZONE"):
    """Calculate the leading scorers by zone

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.

    Returns:
        pd.DataFrame: A dataframe with the leading scorers by zone and their points
            scored in that zone
    """
    top_players_by_zone_df = df.groupby([zone_col, "PLAYER"])["POINTS"].sum().reset_index()
    pidx = top_players_by_zone_df.groupby(zone_col)['POINTS'].idxmax()
    top_players = top_players_by_zone_df.loc[pidx]
    # top_players.rename(columns={zone_col: "ZONE"}, inplace=True)
    return top_players

In [3]:
season = 2024
competition = "E"

In [ ]:
shotdata = ShotData(competition)
shot_df = shotdata.get_game_shot_data_single_season(season)

# Find leading scorers by zone

In [5]:
round_ = None  # Set to None if you want all rounds of the season

In [6]:
shot_df = corner_three_zones(shot_df)
# shot_df = hexagon_zones(shot_df, gridsize=3)
# fg_perc_hex_heatmap(shot_df, gridsize=4)

In [ ]:
if round_ is not None:
   top_players = get_leading_scorer_by_zone(shot_df[(shot_df["Round"] == round_)], zone_col="ZONE_C")
else:
    top_players = get_leading_scorer_by_zone(shot_df, zone_col="ZONE_C")
top_players

In [ ]:
plot_leading_scorers_by_zone(shot_df, top_players, zone_col="ZONE_C")

# Player Shooting Chart

In [11]:
player_name = 'NEDOVIC, NEMANJA'
round_ = 1  # Set to None if you want all rounds of the season

In [ ]:
player_mask = (shot_df["PLAYER"] == player_name)
if round_ is not None:
     player_mask &= (shot_df["Round"] == round_)

player_shot_df = shot_df[shot_df["PLAYER"] == player_name]
palyer_missed_mask = player_shot_df["ID_ACTION"].str.contains("FGA")
player_made_mask = player_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(player_shot_df[player_made_mask], player_shot_df[palyer_missed_mask], title=f'{player_name} - Shot Chart', color="red")

# Teams' shooting chart

In [5]:
from euroleague_api.team_stats import TeamStats

teamstats = TeamStats("E")
team_info = teamstats.get_team_stats_single_season(endpoint="traditional", season=2024, phase_type_code=None, statistic_mode="PerGame")
code_to_name = team_info[["team.code", "team.name"]].set_index("team.code").to_dict()["team.name"]

In [6]:
team_abbr = "MCO"
round_ = None  # Set to None if you want all rounds of the season

In [ ]:
team_mask = (shot_df["TEAM"] == team_abbr)
if round_ is not None:
     team_mask &= (shot_df["Round"] == round_)

team_shot_df = shot_df[team_mask]
team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")

## Plot shooting charts of all teams

In [18]:
round_ = 2  # Set to None if you want all rounds of the season

In [ ]:
for team_abbr in shot_df["TEAM"].unique():
    team_mask = (shot_df["TEAM"] == team_abbr)
    if round_ is not None:
        team_mask &= (shot_df["Round"] == round_)
    team_shot_df = shot_df[team_mask]
    team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
    team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")
    plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")